## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import Callback
# from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras import layers
import keras
from keras.callbacks import TensorBoard
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.shape

(34299, 12)

In [3]:
application_df.sample(10)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
28559,812622905,BENEFITS ASSISTING RETIRING K-9S INC,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
8884,300283263,WILLIE YOUNGS HOME FOR YOUNG MEN INC,T3,Independent,C1000,Preservation,Corporation,1,25000-99999,N,62059,1
20388,510249125,AIR FORCE ASSOCIATION,T3,CompanySponsored,C1000,Preservation,Association,1,0,N,5000,1
18594,473231010,PARENT BOOSTER USA INC,T3,CompanySponsored,C2100,Preservation,Trust,1,0,N,5000,0
26208,760036634,SAMUEL OTERO EVANGELISTIC ASSOCIATION,T3,Independent,C7000,Preservation,Trust,1,0,N,5000,1
26056,753208732,TENNESSEE ORDER OF THE EASTERN STAR,T8,CompanySponsored,C1000,ProductDev,Association,1,0,N,5000,0
19225,474619946,UNITED STATES AIR FORCE,T19,Independent,C1000,ProductDev,Association,1,0,N,5000,1
20403,510256086,LITTLE LEAGUE BASEBALL INC,T3,CompanySponsored,C1000,Preservation,Association,1,25000-99999,N,147646,0
2393,204660123,PTA UTAH CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
25148,742849611,SAINT LUKES HOSPITAL OF GARNETT INC,T3,Independent,C1000,Heathcare,Trust,1,10M-50M,N,6987462,1


#### IS_SUCCESSFUL is the target for our model. 
#### EIN, NAME, APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT are the features of our model.

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df.drop(['EIN','NAME'], axis = 1)
application_df2.shape

(34299, 10)

In [5]:
# Determine the number of unique values in each column.
counts = application_df2.nunique()
counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### For those columns that have more than 10 unique values, determine the number of data points for each unique value.

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df2['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
sr = application_df2['APPLICATION_TYPE'].value_counts()

In [8]:
type(sr)

pandas.core.series.Series

In [9]:
list = sr.index.to_list()
print(list)

['T3', 'T4', 'T6', 'T5', 'T19', 'T8', 'T7', 'T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T15', 'T29', 'T17']


In [10]:
list.remove('T3')
list.remove('T4')
list.remove('T6')
list.remove('T5')
list.remove('T19')
list.remove('T8')
list.remove('T7')
list.remove('T10')
#,'T6','T5','T19','T8','T7','T10')
print(list)

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T15', 'T29', 'T17']


In [11]:
application_types_to_replace = list

In [12]:
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T15', 'T29', 'T17']

In [13]:
# Choose a cutoff value and create a list of application types to be replaced T10 GREATER THAN OR EQUAL TO 528; T9 LESS THAN 528  
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

# Replace in dataframe
for app in application_types_to_replace:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [14]:
application_df2.nunique()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [15]:
# pd.set_option("display.max_rows", None)

In [16]:
# Look at CLASSIFICATION value counts for binning
# application_df2['ASK_AMT'].value_counts() maybe delete this comment if it is not needed?
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C8210        1
C6100        1
C1570        1
C1370        1
C4120        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [17]:
sr2 = application_df2['CLASSIFICATION'].value_counts()
type(sr2)

pandas.core.series.Series

In [18]:
list2 = sr2.index.to_list()
print(list2)

['C1000', 'C2000', 'C1200', 'C3000', 'C2100', 'C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C2300', 'C7200', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1278', 'C1238', 'C1235', 'C1237', 'C7210', 'C1720', 'C4100', 'C2400', 'C1257', 'C1600', 'C1260', 'C0', 'C2710', 'C1234', 'C1267', 'C1246', 'C3200', 'C1256', 'C3700', 'C2570', 'C2170', 'C1236', 'C4500', 'C2500', 'C1580', 'C2190', 'C1728', 'C1248', 'C1732', 'C2600', 'C2380', 'C1820', 'C5200', 'C1900', 'C2150', 'C1245', 'C2561', 'C4200', 'C1283', 'C8210', 'C6100', 'C1570', 'C1370', 'C4120']


In [19]:
list2.remove('C1000')
list2.remove('C2000')
list2.remove('C1200')
list2.remove('C3000')
list2.remove('C2100')
print(list2)

['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C2300', 'C7200', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1278', 'C1238', 'C1235', 'C1237', 'C7210', 'C1720', 'C4100', 'C2400', 'C1257', 'C1600', 'C1260', 'C0', 'C2710', 'C1234', 'C1267', 'C1246', 'C3200', 'C1256', 'C3700', 'C2570', 'C2170', 'C1236', 'C4500', 'C2500', 'C1580', 'C2190', 'C1728', 'C1248', 'C1732', 'C2600', 'C2380', 'C1820', 'C5200', 'C1900', 'C2150', 'C1245', 'C2561', 'C4200', 'C1283', 'C8210', 'C6100', 'C1570', 'C1370', 'C4120']


In [20]:
classifications_to_replace = list2

In [21]:
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C2300',
 'C7200',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1278',
 'C1238',
 'C1235',
 'C1237',
 'C7210',
 'C1720',
 'C4100',
 'C2400',
 'C1257',
 'C1600',
 'C1260',
 'C0',
 'C2710',
 'C1234',
 'C1267',
 'C1246',
 'C3200',
 'C1256',
 'C3700',
 'C2570',
 'C2170',
 'C1236',
 'C4500',
 'C2500',
 'C1580',
 'C2190',
 'C1728',
 'C1248',
 'C1732',
 'C2600',
 'C2380',
 'C1820',
 'C5200',
 'C1900',
 'C2150',
 'C1245',
 'C2561',
 'C4200',
 'C1283',
 'C8210',
 'C6100',
 'C1570',
 'C1370',
 'C4120']

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE

# Replace in dataframe
for cls in classifications_to_replace:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [23]:
application_df2.nunique()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION               6
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [24]:
application_df2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
# APPLICATION_TYPE	AFFILIATION	CLASSIFICATION	USE_CASE	ORGANIZATION
# One-hot encoding the 'workclass' column creates 9 new columns, one for each category
dummies1 = pd.get_dummies(application_df2['APPLICATION_TYPE']) # adds 9 column
dummies2 = pd.get_dummies(application_df2['AFFILIATION']) # adds 6 column
dummies3 = pd.get_dummies(application_df2['CLASSIFICATION']) # adds 6 column
dummies4 = pd.get_dummies(application_df2['USE_CASE']) # adds 5 column
dummies5 = pd.get_dummies(application_df2['ORGANIZATION']) # adds 4 column
dummies6 = pd.get_dummies(application_df2['SPECIAL_CONSIDERATIONS']) # adds 2 columns
dummies7 = pd.get_dummies(application_df2['INCOME_AMT']) # adds 9 columns

In [26]:
merged1 = pd.concat([application_df2, dummies1], axis='columns')
merged1

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,T7,T8
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,0,1,0,0,0,0,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,0,0,0,1,0,0,0,0,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,0,0,0,0,0,1,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,0,0,0,1,0,0,0,0,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,0,0,0,0,1,0,0,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,0,0,0,0,1,0,0,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,0,0,0,1,0,0,0,0,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,0,0,0,0,0,1,0,0,0


In [27]:
merged2 = pd.concat([merged1, dummies2], axis='columns')
merged2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,T5,T6,T7,T8,CompanySponsored,Family/Parent,Independent,National,Other,Regional
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,0,0,0,1,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0,0,0,0,0,0,1,0,0,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,1,0,0,0,1,0,0,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,0,0,0,1,0,0,0,0,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,1,0,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,1,0,0,0,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,0,0,0,1,0,0,0,0,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,1,0,0,0,0,0,1,0,0,0


In [28]:
merged3 = pd.concat([merged2, dummies3], axis='columns')
merged3

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,Independent,National,Other,Regional,C1000,C1200,C2000,C2100,C3000,Other
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,1,0,0,0,1,0,0,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,1,0,0,0,0,0,1,0,0,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,0,0,0,0,0,1,0,0,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,1,0,0,0,1,0,0,0,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,0,0,0,0,0,1,0,0,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,1,0,0,0,0,0,0,0,1,0


In [29]:
merged4 = pd.concat([merged3, dummies4], axis='columns')
merged4

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,C1200,C2000,C2100,C3000,Other,CommunityServ,Heathcare,Other,Preservation,ProductDev
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,0,0,0,0,0,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0,1,0,0,0,0,0,0,1,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,1,0,0,0,0,0,1
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,1,0,0,0,0,0,0,1,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,0,0,0,0,1
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,1,0,0,0,0,0,1
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,1,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,1,0,0,0,0,0,1


In [30]:
merged5 = pd.concat([merged4, dummies5], axis='columns')
merged5

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,Other,CommunityServ,Heathcare,Other,Preservation,ProductDev,Association,Co-operative,Corporation,Trust
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,0,0,1,1,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0,0,0,0,1,0,0,1,0,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,1,1,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,0,0,0,1,0,0,0,0,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,1,1,0,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,0,0,1,1,0,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,0,0,0,1,0,1,0,0,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,0,0,1,1,0,0,0


In [31]:
merged6 = pd.concat([merged5, dummies6], axis='columns')
merged6

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,Heathcare,Other,Preservation,ProductDev,Association,Co-operative,Corporation,Trust,N,Y
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,1,1,0,0,0,1,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0,0,1,0,0,1,0,0,1,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,1,1,0,0,0,1,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,0,1,0,0,0,0,1,1,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,1,1,0,0,0,1,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,0,0,1,1,0,0,0,1,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,0,1,0,1,0,0,0,1,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0,0,0,1,1,0,0,0,1,0


In [32]:
merged7 = pd.concat([merged6, dummies7], axis='columns')
merged7

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,Y,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,1,0,0,0,0,0,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0,0,1,0,0,0,0,0,0,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,0,0,1,0,0,0,0,0,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0,1,0,0,0,0,0,0,0,0


In [33]:
newdf = merged7.drop(['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], axis = 1)
# APPLICATION_TYPE	AFFILIATION	CLASSIFICATION	USE_CASE	ORGANIZATION

In [34]:
newdf

,STATUS,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,...,Y,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [35]:
# Split our preprocessed data into our features and target arrays
y = newdf["IS_SUCCESSFUL"].values
X = newdf.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

In [37]:
print(X_train)

[[      1    5000       0 ...       0       0       0]
 [      1    5000       0 ...       0       0       0]
 [      1    5000       0 ...       0       0       0]
 ...
 [      1  103405       0 ...       1       0       0]
 [      1    5000       0 ...       0       0       0]
 [      1 1869240       0 ...       0       0       0]]


In [38]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [39]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [40]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 8
hidden_nodes_layer6 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 352       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 8)                 72        
                                                                 
 dense_6 (Dense)             (None, 1)                 9

In [41]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [42]:
mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)

In [43]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50, callbacks=[mc])
# fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 718us/step - loss: 0.6109 - accuracy: 0.6973
Epoch 2/50
804/804 [==============================] - 1s 718us/step - loss: 0.5683 - accuracy: 0.7287
Epoch 3/50
804/804 [==============================] - 1s 736us/step - loss: 0.5615 - accuracy: 0.7302
Epoch 4/50
804/804 [==============================] - 1s 719us/step - loss: 0.5562 - accuracy: 0.7310
Epoch 5/50
804/804 [==============================] - 1s 695us/step - loss: 0.5539 - accuracy: 0.7312
Epoch 6/50
804/804 [==============================] - 1s 699us/step - loss: 0.5527 - accuracy: 0.7298
Epoch 7/50
804/804 [==============================] - 1s 700us/step - loss: 0.5521 - accuracy: 0.7309
Epoch 8/50
804/804 [==============================] - 1s 698us/step - loss: 0.5515 - accuracy: 0.7322
Epoch 9/50
804/804 [==============================] - 1s 701us/step - loss: 0.5509 - accuracy: 0.7317
Epoch 10/50
804/804 [==============================] - 1s 707us/step - loss: 0.549

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5520 - accuracy: 0.7258 - 221ms/epoch - 824us/step
Loss: 0.5520287156105042, Accuracy: 0.7258309125900269


In [45]:
# Export our model to HDF5 file 
nn.save('AlphabetSoupCharity_Optimization3.h5')

In [46]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model('AlphabetSoupCharity_Optimization3.h5')

In [47]:
# Checking:
np.testing.assert_allclose(
    nn.predict(X_train_scaled), reconstructed_model.predict(X_train_scaled)
)

In [48]:
# The reconstructed model is already compiled and has retained the optimizer state, so training can resume:
reconstructed_model.fit(X_train_scaled, y_train)

804/804 [==============================] - 1s 974us/step - loss: 0.5448 - accuracy: 0.7346
